# Fuzzy Table Match

You have two tables that you want to match. You are not sure which columns should match with which column, and you also want the match to tolerate minor differences and variations to find close matches rather than exact matches. 

Tasks achieved here 💃🏻

* ✅ Compare each row in table 1 with each row in table 2 and return a match score for each combination (which is the total # of columns matched)
* ✅ Return a joined table with all matched rows where match score > 0
* ✅ Incorporating fuzzy match for all the string columns 

## Matching two number tables

In [95]:
import pandas as pd
from IPython.display import display

table1 = pd.DataFrame({
    'A': [1, 2],
    'B': [3, 5],
})


table2 = pd.DataFrame({
    'X': [2, 1, 1],
    'Y': [5, 7, 3]
})

In [96]:
# Initialize an empty list to store matching results
matching_results = []

# Loop through each row in table1
for _, row1 in table1.iterrows():
    # Initialize a list to store matching rows and their scores
    matches = []
    
    # Loop through each row in table2
    for _, row2 in table2.iterrows():
        
        # Count the number of matching columns
        match_score = sum(row1[col1] == row2[col2] for col1 in table1.columns for col2 in table2.columns)

        # If there's at least one matching column
        if match_score > 0:
            # Merge the dictionaries to form a new row containing all columns
            matched_row = {**row1, **row2}
            # Add match score to the new row
            matched_row['match_score'] = match_score
            # Append the matched row to matches
            matches.append(matched_row)
    
    # If there are matches for the current row in table1
    if matches:
        # Extend matching_results with matches
        matching_results.extend(matches)

# Create a DataFrame from the list of matching results
matching_results_df = pd.DataFrame(matching_results)
matching_results_df = matching_results_df.sort_values(by = ['A', 'B', 'match_score'], ascending=[True, True, False])

In [97]:
print("Table 1:")
display(table1)
print("\nTable 2:")
display(table2)
print("\nMatching Results:")
display(matching_results_df)

Table 1:


,A,B
0,1,3
1,2,5



Table 2:


,X,Y
0,2,5
1,1,7
2,1,3



Matching Results:


,A,B,X,Y,match_score
1,1,3,1,3,2
0,1,3,1,7,1
2,2,5,2,5,2


## Matching tables of numbers and strings 

In [98]:
# Sample DataFrames with mixed numeric and categorical columns
table1 = pd.DataFrame({
    'A': [1, 2],
    'B': ['cat', 'bird'],  
})

table2 = pd.DataFrame({
    'X': [1, 2, 1],
    'Y': ['dog', 'birdy', 'cat'], 
})

In [99]:
# Initialize an empty list to store matching results
matching_results = []

# Loop through each row in table1
for _, row1 in table1.iterrows():
    # Initialize a list to store matching rows and their scores
    matches = []
    
    # Loop through each row in table2
    for _, row2 in table2.iterrows():
        # Initialize match score
        match_score = 0
        
        # Check numeric columns for equality
        for col1, col2 in zip(table1.select_dtypes(include=['number']), table2.select_dtypes(include=['number'])):
            if row1[col1] == row2[col2]:
                match_score += 1
        
        # Check categorical columns for equality (convert to string for comparison)
        for col1, col2 in zip(table1.select_dtypes(exclude=['number']), table2.select_dtypes(exclude=['number'])):
            if str(row1[col1]) == str(row2[col2]):
                match_score += 1
        
        # If there's at least one matching column
        if match_score > 0:
            # Merge the dictionaries to form a new row containing all columns
            matched_row = {**row1, **row2}
            # Add match score to the new row
            matched_row['match_score'] = match_score
            # Append the matched row to matches
            matches.append(matched_row)
    
    # If there are matches for the current row in table1
    if matches:
        # Extend matching_results with matches
        matching_results.extend(matches)

# Create a DataFrame from the list of matching results
matching_results_df = pd.DataFrame(matching_results)
matching_results_df = matching_results_df.sort_values(by = ['A', 'B', 'match_score'], ascending=[True, True, False])

In [100]:
print("Table 1:")
display(table1)
print("\nTable 2:")
display(table2)
print("\nMatching Results:")
display(matching_results_df)

Table 1:


,A,B
0,1,cat
1,2,bird



Table 2:


,X,Y
0,1,dog
1,2,birdy
2,1,cat



Matching Results:


,A,B,X,Y,match_score
1,1,cat,1,cat,2
0,1,cat,1,dog,1
2,2,bird,2,birdy,1


## Introducing Fuzzy Match

In [101]:
# Sample DataFrames with mixed numeric and categorical columns
table1 = pd.DataFrame({
    'A': [1, 2],
    'B': ['cat', 'bird'],  
})

table2 = pd.DataFrame({
    'X': [1, 2, 1],
    'Y': ['dog', 'birdy', 'cat'], 
})

In [102]:
from fuzzywuzzy import fuzz  

# Initialize an empty list to store matching results
matching_results = []

# Loop through each row in table1
for _, row1 in table1.iterrows():
    # Initialize a list to store matching rows and their scores
    matches = []
    
    # Loop through each row in table2
    for _, row2 in table2.iterrows():
        # Initialize match score
        match_score = 0
        
        # Check numeric columns for equality
        for col1, col2 in zip(table1.select_dtypes(include=['number']), table2.select_dtypes(include=['number'])):
            if row1[col1] == row2[col2]:
                match_score += 1
        
        # Check string columns for similarity (fuzzy match)
        for col1, col2 in zip(table1.select_dtypes(exclude=['number']), table2.select_dtypes(exclude=['number'])):
            similarity_score = fuzz.ratio(str(row1[col1]), str(row2[col2]))
            if similarity_score >= 80:  # Adjust the threshold as needed
                match_score += 1
        
        # If there's at least one matching column
        if match_score > 0:
            # Merge the dictionaries to form a new row containing all columns
            matched_row = {**row1, **row2}
            # Add match score to the new row
            matched_row['match_score'] = match_score
            # Append the matched row to matches
            matches.append(matched_row)
    
    # If there are matches for the current row in table1
    if matches:
        # Extend matching_results with matches
        matching_results.extend(matches)

# Create a DataFrame from the list of matching results
matching_results_df = pd.DataFrame(matching_results)
matching_results_df = matching_results_df.sort_values(by = ['A', 'B', 'match_score'], ascending=[True, True, False])

In [103]:
print("Table 1:")
display(table1)
print("\nTable 2:")
display(table2)
print("\nMatching Results:")
display(matching_results_df)

Table 1:


,A,B
0,1,cat
1,2,bird



Table 2:


,X,Y
0,1,dog
1,2,birdy
2,1,cat



Matching Results:


,A,B,X,Y,match_score
1,1,cat,1,cat,2
0,1,cat,1,dog,1
2,2,bird,2,birdy,2
